In [2]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math

In [39]:
## Loading Constants
start = "2008-01-01"
end = datetime.now().strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("recs")
market = Market()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500")
market.close()
reload = False
current_week = int(datetime.now().strftime("%W")) + 1
current_quarter = (datetime.now().month - 1)//3 +1
current_year = datetime.now().year
model_range = range(len(tickers["Symbol"]))
weekly_range = range(current_week - 1,current_week)
weekly_range_final = range(current_week - 15,current_week)
quarterly_range = range(current_quarter-2,current_quarter)
quarterly_fundamental_range = range(current_quarter-1,current_quarter +1)
yearly_fundamental_range = range(current_year-1,current_year)
yearly_range = range(current_year,current_year +1)
ticker_range = range(len(tickers["Symbol"]))
tier_2_yearly_range = range(current_year,current_year+1)
load_quarter = False

In [20]:
refined_regression = []
refined_daily_classification = []
refined_quarterly_classification = []
refined_weekly_classification = []
market.connect()
if reload:
    for i in tqdm(ticker_range):
        try:
            ticker = tickers["Symbol"][i]
            if "." in ticker:
                ticker = ticker.replace(".","-")
            prices = market.retrieve_price_data("prices",ticker)
            prices.reset_index(inplace=True)
            relev = prices[["date","adjClose"]]
            for column in ["date","adjClose"]:
                relev.rename(columns={column:"".join(column.split("_")).lower()},inplace=True)
            relev["date"] = [datetime.strptime(x.split("T")[0],"%Y-%m-%d") for x in relev["date"]]
            relev["ticker"] = ticker
            relev.sort_values("date",inplace=True)
            ## daily transformations
            refined_regression.append(relev)
            relev_classification = relev.copy()
            relev_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_classification["adjclose"].diff()]
            refined_daily_classification.append(relev_classification)
            ## weekly transformations
            relev["week"] = [x.week for x in relev["date"]]
            relev["year"] = [x.year for x in relev["date"]]
            relev["quarter"] = [x.quarter for x in relev["date"]]
            relev_weekly_classification = relev.groupby(["year","week"]).mean().reset_index()
            relev_weekly_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_weekly_classification["adjclose"].diff()]
            relev_weekly_classification["ticker"] = ticker
            refined_weekly_classification.append(relev_weekly_classification)
            ## quarterly transformations
            relev_quarterly_classification = relev.groupby(["year","quarter"]).mean().reset_index().drop("week",axis=1)
            relev_quarterly_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_quarterly_classification["adjclose"].diff()]
            relev_quarterly_classification["ticker"] = ticker
            refined_quarterly_classification.append(relev_quarterly_classification)
        except Exception as e:
            print(str(e),ticker)
market.close()
classification_sets = {"date":refined_daily_classification,
                       "quarter":refined_quarterly_classification,
                      "week":refined_weekly_classification}

In [21]:
if reload:
    for dataset in classification_sets:
        base = pd.concat(classification_sets[dataset])
        if dataset == "date":
            final = base.pivot_table(index=dataset,values="adjclose",columns="ticker").reset_index()
            final = final[final["date"] >= datetime.strptime(start,"%Y-%m-%d")]
        else:
            final = base.pivot_table(index=["year",dataset],values="adjclose",columns="ticker").reset_index()
            final = final[final["year"] >= datetime.strptime(start,"%Y-%m-%d").year]
        name = "dataset_{}_classification".format(dataset)
        for column in final.columns:
            if "_" in column:
                final.drop(column,axis=1,inplace=True,errors="ignore")
        final.fillna(-99999,inplace=True)
        for column in tqdm(final.columns):
            if -99999 == final[column].min():
                final = final.drop(column,axis=1)
        market.drop_table(name)
        market.store_data(name,final)

In [22]:
if reload:
    base = pd.concat(refined_regression)
    final = base.pivot_table(index="date",values="adjclose",columns="ticker").reset_index()
    final = final[final["date"] >= datetime.strptime(start,"%Y-%m-%d")]
    for column in final.columns:
        if "_" in column:
            final.drop(column,axis=1,inplace=True,errors="ignore")
    final.fillna(-99999,inplace=True)
    for column in tqdm(final.columns):
        if -99999 == final[column].min():
            final.drop(column,axis=1,inplace=True)
    market.drop_table("dataset_date_regression")
    market.store_data("dataset_date_regression",final)
    final["quarter"] = [x.quarter for x in final["date"]]
    final["year"] = [x.year for x in final["date"]]
    final["week"] = [x.week for x in final["date"]]
    for timeframe in ["week","quarter"]:
        relev = final.groupby(["year",timeframe]).mean().reset_index()
        relev.reset_index(drop=True,inplace=True)
        name = "dataset_{}_regression".format(timeframe)
        market.drop_table(name)
        market.store_data(name,relev)

In [21]:
sims = []
training_days = 100
gap = 90
quarter_gap = int(gap/90)
training_years = 4
training_days = 100
sims = []
sec.connect()
strat_db.connect()
market.connect()
quarterly_classification_data = market.retrieve_data("dataset_quarter_classification")
quarterly_regression_data = market.retrieve_data("dataset_quarter_regression")
price_regression_data = market.retrieve_data("dataset_date_regression")
if load_quarter:
    for year in tqdm(yearly_range):
        for quarter in tqdm(quarterly_range):
            try:
                ## Setting Up
                print(year,quarter)
                date_ranges = DateUtils.create_quarterly_training_range_rec(year,quarter,training_years,gap)
                training_start,training_end,prediction_start,prediction_end = date_ranges
                dates = pd.to_datetime(date_ranges)
                quarters = [x.quarter for x in dates]
                years = [x.year for x in dates]
                print(date_ranges)
                ### switch to all tickers
                for i in model_range:
                    try:
                        ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                        if ticker in quarterly_regression_data.columns:
                            price = market.retrieve_price_data("prices",ticker)
                            mt = ModelTransformer(ticker,training_start,training_end,gap)
                            mr = ModelPreprocessor(ticker)
                            sp = Modeler(ticker)
                            prot = ProductTransformer(ticker,prediction_start,prediction_end)
                            ## regression_model
                            rqpd = mt.quarterly_price_transform(quarterly_regression_data,ticker
                                                                                ,years[0],quarters[0],
                                                                               years[1],quarters[1],quarter_gap)
                            qpd = mr.day_trade_preprocess_regression(rqpd.copy(),ticker)
                            q1c = qpd["X"].columns
                            rpr = sp.quarterly_model("prices",qpd,str(training_years),str(gap))
                            ## classification_model
                            cqpd = mt.quarterly_price_transform(quarterly_classification_data,ticker
                                                                                ,years[0],quarters[0],
                                                                               years[1],quarters[1],quarter_gap)
                            qpd = mr.day_trade_preprocess_classify(cqpd.copy(),ticker)
                            q2c = qpd["X"].columns
                            cpr = sp.classify_wta("prices",qpd,str(training_years),str(gap))
                            price_results = pd.DataFrame([cpr,rpr])
                            product_refineds = []
                            product_qpds = []
                            for i in range(len(price_results)):
                                price_result = price_results.iloc[i]
                                if price_result["classification"] == False:
                                    quarterly_price_data = quarterly_regression_data
                                else:
                                    quarterly_price_data = quarterly_classification_data
                                quarterly_price_data = quarterly_price_data[(quarterly_price_data["year"] == years[3]) & 
                                                                           (quarterly_price_data["quarter"] == quarters[3])]
                                quarterly_price_data["y"] = quarterly_price_data[ticker]
                                product_qpd = mr.day_trade_preprocess_regression(quarterly_price_data.copy(),ticker)
                                product_qpds.append(product_qpd)
                            sim = prot.merge_quarterlies_price(price.copy(),product_qpds,price_results)
                            strat_db.store_data("quarterly_sim",sim)
                    except Exception as e:
                        message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                        print(message)
            except Exception as e:
                print(year,quarter,str(e))
sec.close()
market.close()
strat_db.close()

In [24]:
sims = []
training_days = 100
gap = 365
quarter_gap = int(gap/90)
training_years = 4
training_days = 100
sims = []
sec.connect()
strat_db.connect()
market.connect()
quarterly_classification_data = market.retrieve_data("dataset_quarter_classification")
quarterly_regression_data = market.retrieve_data("dataset_quarter_regression")
price_regression_data = market.retrieve_data("dataset_date_regression")
if load_quarter:
    for year in tqdm(yearly_fundamental_range):
        for quarter in tqdm(quarterly_fundamental_range):
            try:
                ## Setting Up
                print(year,quarter)
                date_ranges = DateUtils.create_quarterly_training_range_rec(year,quarter,training_years,gap)
                training_start,training_end,prediction_start,prediction_end = date_ranges
                dates = pd.to_datetime(date_ranges)
                quarters = [x.quarter for x in dates]
                years = [x.year for x in dates]
                ### switch to all tickers
                for i in model_range:
                    try:
                        ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                        if ticker in quarterly_regression_data.columns:
                            cik = int(tickers.iloc[i]["CIK"].item())
                            price = market.retrieve_price_data("prices",ticker)
                            filing = sec.retrieve_filing_data(cik)
                            mt = ModelTransformer(ticker,training_start,training_end,gap)
                            mr = ModelPreprocessor(ticker)
                            sp = Modeler(ticker)
                            prot = ProductTransformer(ticker,prediction_start,prediction_end)
                            ## regression_model
                            rfd = mt.fundamental_merge(price.copy(),filing.copy(),True,classify=False)
                            refined = mr.fundamental_preprocess(rfd.copy())
                            rfr = sp.quarterly_model("prices",refined,str(training_years),str(gap))
                            ## classification_model
                            cfd = mt.fundamental_merge(price.copy(),filing.copy(),True,classify=True)
                            refined = mr.fundamental_preprocess(cfd.copy())
                            cfr = sp.classify_wta("prices",refined,str(training_years),str(gap))
                            fundamental_results = pd.DataFrame([cfr,rfr])
                            product_refineds = []
                            product_qpds = []
                            for i in range(len(fundamental_results)):
                                fundamental_result = fundamental_results.iloc[i]
                                if fundamental_result["classification"] == False:
                                    fundamental_data = rfd
                                else:
                                    fundamental_data = cfd
                                product_refined = mr.fundamental_preprocess(fundamental_data.copy())
                                product_refineds.append(product_refined)
                            sim = prot.merge_quarterlies_fundamental(price.copy(),product_refineds,fundamental_results)
                            strat_db.store_data("fundamental_quarterly_sim",sim)
                    except Exception as e:
                        message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                        print(message)
            except Exception as e:
                print(year,quarter,str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                         | 0/2 [00:00<?, ?it/s]

2019 3



 50%|███████████████████████████████████████████████████████▌                                                       | 1/2 [17:57<17:57, 1077.57s/it]

2019 4



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [35:23<00:00, 2123.28s/it]


In [26]:
sims = []
test = True
training_days = 100
gap = 5
week_gap = int(gap/5)
training_years = 4
training_days = 100
daily_gap = 2
timeline = DateUtils.create_timeline(start,end)
sims = []
sec.connect()
strat_db.connect()
market.connect()
classification = market.retrieve_data("dataset_week_classification")
regression = market.retrieve_data("dataset_week_regression")
for year in tqdm(yearly_range):
    for week in tqdm(weekly_range):
        try:
            ## Setting Up
            date_ranges = DateUtils.create_weekly_training_range_rec(timeline,year,week,training_years)
            training_start,training_end,prediction_start,prediction_end = date_ranges
            dates = pd.to_datetime(date_ranges)
            weeks = [x.week for x in dates]
            years = [x.year for x in dates]
            for i in model_range:
                try:
                    ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                    if ticker in regression.columns:
                        price = market.retrieve_price_data("prices",ticker)
                        mt = ModelTransformer(ticker,training_start,training_end,week_gap)
                        mr = ModelPreprocessor(ticker)
                        sp = Modeler(ticker)
                        prot = ProductTransformer(ticker,prediction_start,prediction_end)
                        ## regression_model
                        rqpd = mt.weekly_price_transform(regression,ticker
                                                                            ,years[0],weeks[0],
                                                                           years[1],weeks[1],week_gap)
                        qpd = mr.day_trade_preprocess_regression(rqpd.copy(),ticker)
                        q1c = qpd["X"].columns
                        rpr = sp.quarterly_model("prices",qpd,str(training_years),str(week_gap))
                        ## classification_model
                        cqpd = mt.weekly_price_transform(classification,ticker
                                                                            ,years[0],weeks[0],
                                                                           years[1],weeks[1],week_gap)
                        qpd = mr.day_trade_preprocess_classify(cqpd.copy(),ticker)
                        q2c = qpd["X"].columns
                        cpr = sp.classify_wta("prices",qpd,str(training_years),str(week_gap))
                        price_results = pd.DataFrame([cpr,rpr])
                        product_qpds = []
                        for i in range(len(price_results)):
                            price_result = price_results.iloc[i]
                            if price_result["classification"] == False:
                                weekly_price_data = regression
                            else:
                                weekly_price_data = classification
                            weekly_price_data = weekly_price_data[(weekly_price_data["year"] == years[3]) & 
                                                                       (weekly_price_data["week"] == weeks[3])]
                            weekly_price_data["y"] = weekly_price_data[ticker]
                            product_qpd = mr.day_trade_preprocess_regression(weekly_price_data.copy(),ticker)
                            product_qpds.append(product_qpd)
                            print(week,year,current_week,current_year)
                        if week == datetime.now().isocalendar()[1] - 1 and year == datetime.now().year:
                            sim = prot.merge_weeklies_predict(product_qpds,price_results)
                            strat_db.store_data("prediction_sim",pd.DataFrame([sim]))
                        else:
                            sim = prot.merge_weeklies(price.copy(),product_qpds,price_results)
                            if len(sim) > 0:
                                strat_db.store_data("weekly_sim",sim)
                except Exception as e:
                    message = {"status":"weekly modeling","ticker":ticker,"year":str(year),"week":str(week),"message":str(e)}
                    print(message)
        except Exception as e:
            print(year,week,str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                         | 0/1 [00:00<?, ?it/s]

43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 

43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 44 2020
43 2020 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [13:42<00:00, 822.11s/it]

43 2020 44 2020
43 2020 44 2020


In [27]:
strat_db.connect()
weekly_sim = strat_db.retrieve_data("weekly_sim")
price_quarterly_sim = strat_db.retrieve_data("quarterly_sim")
fundamental_quarterly_sim = strat_db.retrieve_data("fundamental_quarterly_sim")
strat_db.close()

In [28]:
quarterly_sim = fundamental_quarterly_sim.drop(["_id","adjclose"],axis=1) \
                    .merge(price_quarterly_sim.drop("_id",axis=1),on=["date","ticker"],how="left")

In [29]:
weekly_sim["passed"] = weekly_sim["wmtd"] > weekly_sim["weekly_price_regression_prediction"]
quarterly_sim_columns = quarterly_sim.columns
weekly_sim_columns = weekly_sim.columns
qsc = [x for x in quarterly_sim_columns if "score" in x]
wsc = [x for x in weekly_sim_columns if "score" in x]
qcc = [x for x in quarterly_sim_columns if "classification_prediction" in x]
wcc = [x for x in weekly_sim_columns if "classification_prediction" in x]
qrc = [x for x in quarterly_sim_columns if "regression_prediction" in x]
wrc = [x for x in weekly_sim_columns if "regression_prediction" in x]

In [30]:
availability = []
sp5 = tickers
end_datetime = datetime.strptime(end,"%Y-%m-%d")
for i in model_range:
    ticker = list(sp5["Symbol"])[i]
    ticker = ticker.replace(".","-")
    td = weekly_sim[(weekly_sim["ticker"]==ticker) & (weekly_sim["date"]<=end_datetime)]
    try:
        availability.append({"ticker":ticker
                             ,"end_date":end_datetime <= td.iloc[len(td)-1]["date"]
                             ,"days":(td.iloc[len(td)-1]["date"]- td.iloc[0]["date"]).days
                            ,"records":len(td[td["date"] <= end_datetime])
                            ,"pct":math.ceil(len(td[td["date"] <= end_datetime])/(end_datetime - td.iloc[0]["date"]).days * 100)})
    except Exception as e:
        print(ticker,str(e))
        continue
stuff = pd.DataFrame(availability)
sp5_tickers = list(stuff[(stuff["pct"] >= stuff["pct"].mode().item() - 1) | (stuff["pct"] <= stuff["pct"].mode().item() + 1)]["ticker"].values)

ABBV single positional indexer is out-of-bounds
ALLE single positional indexer is out-of-bounds
GOOG single positional indexer is out-of-bounds
AMCR single positional indexer is out-of-bounds
AWK single positional indexer is out-of-bounds
APTV single positional indexer is out-of-bounds
ANET single positional indexer is out-of-bounds
BKR single positional indexer is out-of-bounds
AVGO single positional indexer is out-of-bounds
CARR single positional indexer is out-of-bounds
CTLT single positional indexer is out-of-bounds
CBOE single positional indexer is out-of-bounds
CBRE single positional indexer is out-of-bounds
CDW single positional indexer is out-of-bounds
CHTR single positional indexer is out-of-bounds
CFG single positional indexer is out-of-bounds
CTVA single positional indexer is out-of-bounds
FANG single positional indexer is out-of-bounds
DISCK single positional indexer is out-of-bounds
DG single positional indexer is out-of-bounds
DOW single positional indexer is out-of-bound

In [41]:
sims = []
test = True
gap = 5
week_gap = int(gap/5)
training_years = 1
training_days = 100
timeline = DateUtils.create_timeline(start,end)
sims = []
sec.connect()
strat_db.connect()
market.connect()
for year in tqdm(tier_2_yearly_range):
    for week in tqdm(weekly_range_final):
        try:
            ## Setting Up
            date_ranges = DateUtils.create_weekly_training_range_rec(timeline,year,week,training_years)
            training_start,training_end,prediction_start,prediction_end = date_ranges
            print(date_ranges)
            for i in range(len(sp5_tickers)):
                try:
                    ticker = sp5_tickers[i].replace(".","-")
                    quarter_sim = quarterly_sim[(quarterly_sim["ticker"] == ticker) & (quarterly_sim["date"] <= prediction_end) & (quarterly_sim["date"] >= training_start)]
                    week_sim = weekly_sim[(weekly_sim["ticker"] == ticker) & (weekly_sim["date"] <= prediction_end) & (weekly_sim["date"] >= training_start)]
                    for col in wrc:
                        week_sim["{}_delta".format(col)] = (week_sim[col] - week_sim["adjclose"])/week_sim["adjclose"]
                    for col in qrc:
                        quarter_sim["{}_delta".format(col)] = (quarter_sim[col] - quarter_sim["adjclose"])/quarter_sim["adjclose"]
                    sim = week_sim.drop("_id",axis=1).merge(quarter_sim.drop("adjclose",axis=1),on=["ticker","date"],how="left").dropna()
                    training_set = sim[sim["date"] <= training_end]
                    prediction_set = sim[sim["date"] >= prediction_start].iloc[:5]
                    try:
                        mr = ModelPreprocessor(ticker)
                        sp = Modeler(ticker)
                        mt = ModelTransformer(ticker,start,end,week_gap)
                        ## classification_model
                        ts = mt.trade_signal_transform_classify(training_set.copy())
                        qpd = mr.trade_signal_preprocess_classify(ts.copy())
                        price_result = sp.classify_wta("prices",qpd,str(training_years),str(week_gap))
                    except Exception as e:
                        message = {"status":"modeling","ticker":ticker,"message":str(e)}
                        print(message)
                    try:
                        if week == datetime.now().isocalendar()[1] - 1 and year == datetime.now().year:
                            strat_db.connect()
                            predicts = strat_db.retrieve_data("prediction_sim")
                            strat_db.close()
                            current_values = sim.tail(1)
                            cols = []
                            for x in current_values.columns:
                                if "weekly" in x.split("_") or "x" in  x.split("_") or "delta" in  x.split("_"):
                                    cols.append(x)
                            current_values.drop(cols,axis=1,inplace=True)
                            for col in predicts:
                                current_values[col] = predicts[col]
                            current_values.drop(["date","ticker","wmtd","adjclose","_id"],axis=1,inplace=True,errors="ignore")
                            ## get adjcloses
                            adjclose = market.retrieve_price_data("prices",ticker).tail(1)["adjClose"].item()
                            wmtd = market.retrieve_price_data("prices",ticker).iloc[:5]["adjClose"].max()
                            current_values["adjclose"] = adjclose
                            current_values["wmtd"] = wmtd
                            current_values["passed_x"] = current_values["wmtd"].item() > current_values["weekly_price_regression_prediction"].item()
                            current_values["quarterly_fundamental_regression_prediction_delta"] = (current_values["quarterly_fundamental_regression_prediction"] - current_values["adjclose"]) / current_values["adjclose"]  
                            current_values["quarterly_price_regression_prediction_delta"] = (current_values["quarterly_price_regression_prediction"] - current_values["adjclose"]) / current_values["adjclose"]
                            current_values["weekly_price_regression_prediction_delta"] = (current_values["weekly_price_regression_prediction"] - current_values["adjclose"]) / current_values["adjclose"]
                            classification_predictions = price_result["model"].predict(current_values)
                            product = current_values
                            product["trade_signal_prediction"] = classification_predictions[0]
                            product["trade_signal_prediction_score"] = price_result["score"].item()
                            product["ticker"] = ticker
                            product["date"] = datetime.now()
                            if product.index.size > 0:
                                strat_db.store_data("recs",product)
                        else:
                            weekly_price_data = prediction_set.copy()
                            weekly_price_data["y"] = weekly_price_data["adjclose"]
                            product_qpd = mr.trade_signal_preprocess_classify(weekly_price_data.copy())
                            classification_predictions = price_result["model"].predict(product_qpd["X"])
                            prediction_set["trade_signal_classification_prediction"] = classification_predictions
                            prediction_set["trade_signal_classification_score"] = price_result["score"]
                            historical = prediction_set[["date","ticker"
                                                      ,"adjclose","weekly_price_regression_prediction"
                                                      ,"weekly_price_regression_prediction_delta"
                                                      ,"weekly_price_regression_score","trade_signal_classification_prediction"
                                                      ,"trade_signal_classification_score"]]
                            if product.index.size > 0:
                                strat_db.store_data("sim",historical)
                    except Exception as e:
                        message = {"status":"packaging","ticker":ticker,"message":str(e)}
                        print(message)
                except Exception as e:
                    message = {"status":"weekly modeling","ticker":ticker,"message":str(e)}
                    print(message)
        except Exception as e:
            print(year,week,str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                        | 0/15 [00:00<?, ?it/s]

['2019-07-15', '2020-07-13', '2020-07-13', '2020-07-17']
{'status': 'modeling', 'ticker': 'MMM', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'MMM', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling', 'ticker': 'ABT', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'ABT', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling', 'ticker': 'ABMD', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'ABMD', 'message': 'Found array with 0 sample(s) (shape=(0, 20))

{'status': 'modeling', 'ticker': 'ABC', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'ABC', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling', 'ticker': 'AME', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'AME', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling', 'ticker': 'AMGN', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'AMGN', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling

{'status': 'modeling', 'ticker': 'BBY', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'BBY', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling', 'ticker': 'BIO', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'BIO', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling', 'ticker': 'BIIB', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'BIIB', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling

{'status': 'modeling', 'ticker': 'CHD', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'CHD', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling', 'ticker': 'CI', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'CI', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling', 'ticker': 'CINF', 'message': 'With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'packaging', 'ticker': 'CINF', 'message': 'Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required.'}
{'status': 'modeling',

...


 87%|███████████████████████████████████████████████████████████████████████████████████████████████▎              | 13/15 [27:09<05:03, 151.76s/it]

['2019-10-14', '2020-10-12', '2020-10-12', '2020-10-16']



 93%|██████████████████████████████████████████████████████████████████████████████████████████████████████▋       | 14/15 [29:51<02:34, 154.84s/it]

['2019-10-21', '2020-10-19', '2020-10-19', '2020-10-23']



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [49:10<00:00, 2950.17s/it]
